In [6]:
import mediapipe as mp
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from ultralytics import YOLO
from transformers import CLIPProcessor
from models.models import EmotionMamba, PersonalityMamba, FusionTransformer
from data_loading.feature_extractor import PretrainedImageEmbeddingExtractor
from utils.config_loader import ConfigLoader

def draw_box(image, box, color=(255, 0, 255)):
    """Draw a rectangle on the image."""
    line_width = 2
    lw = line_width or max(round(sum(image.shape) / 2 * 0.003), 2)
    p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
    cv2.rectangle(image, p1, p2, color, thickness=lw, lineType=cv2.LINE_AA)

def image_processing(image, image_processor):
    image = image_processor(images=image, return_tensors="pt").to("cuda")
    image = image['pixel_values']
    return image

def preprocess_face(face_roi: np.ndarray) -> np.ndarray:
    """Предобработка области лица (пример: нормализация + resize)."""
    # Пример: преобразуем в 112x112 и нормализуем [0, 1]
    face_roi = cv2.resize(face_roi, (112, 112))
    face_roi = face_roi.astype('float32') / 255.0
    return face_roi

def preprocess_body(body_roi: np.ndarray) -> np.ndarray:
    """Предобработка области тела (пример: нормализация + resize)."""
    # Пример: преобразуем в 224x224 и нормализуем [0, 1]
    body_roi = cv2.resize(body_roi, (224, 224))
    body_roi = body_roi.astype('float32') / 255.0
    return body_roi

def select_uniform_frames(frames, N):
    if len(frames) <= N:
        return frames
    else:
        indices = np.linspace(0, len(frames) - 1, num=N, dtype=int)
        return [frames[i] for i in indices]

def get_fusion_model(config, device):
    emo_model = EmotionMamba(
    input_dim_emotion     = config.image_embedding_dim,
    input_dim_personality = config.image_embedding_dim,
    len_seq               = config.counter_need_frames, 
    hidden_dim            = config.hidden_dim_emo,
    out_features          = config.out_features_emo,
    tr_layer_number       = config.tr_layer_number_emo,
    num_transformer_heads = config.num_transformer_heads_emo,
    positional_encoding   = config.positional_encoding_emo,
    mamba_d_model         = config.mamba_d_state_emo,
    mamba_layer_number    = config.mamba_layer_number_emo,
    dropout               = config.dropout,
    num_emotions          = 7,
    num_traits            = 5,
    device                = device
    ).to(device).eval()
    # параметры задаем для лучшей персональной модели
    per_model = PersonalityMamba(
    input_dim_emotion     = config.image_embedding_dim,
    input_dim_personality = config.image_embedding_dim,
    len_seq               = config.counter_need_frames, 
    hidden_dim            = config.hidden_dim_per,
    out_features          = config.out_features_per,
    per_activation        = config.best_per_activation,
    tr_layer_number       = config.tr_layer_number_per,
    num_transformer_heads = config.num_transformer_heads_per,
    positional_encoding   = config.positional_encoding_per,
    mamba_d_model         = config.mamba_d_state_per,
    mamba_layer_number    = config.mamba_layer_number_per,
    dropout               = config.dropout,
    num_emotions          = 7,
    num_traits            = 5,
    device                = device
    ).to(device).eval()

    # emo_state = torch.load(config.path_to_saved_emotion_model, map_location=device)
    # emo_model.load_state_dict(emo_state)

    # emo_state = torch.load(config.path_to_saved_personality_model, map_location=device)
    # per_model.load_state_dict(emo_state)
    model = FusionTransformer(
        emo_model             = emo_model,
        per_model             = per_model,
        input_dim_emotion     = config.image_embedding_dim,
        input_dim_personality = config.image_embedding_dim,
        hidden_dim            = config.hidden_dim,
        out_features          = config.out_features,
        per_activation        = config.per_activation,
        tr_layer_number       = config.tr_layer_number,
        num_transformer_heads = config.num_transformer_heads,
        positional_encoding   = config.positional_encoding,
        mamba_d_model         = config.mamba_d_state,
        mamba_layer_number    = config.mamba_layer_number,
        dropout               = config.dropout,
        num_emotions          = 7,
        num_traits            = 5,
        device                = device
        ).to(device).eval()

    return model

def transform_matrix(matrix):
    threshold1 = 1 - 1/7 
    threshold2 = 1/7
    mask1 = matrix[:, 0] >= threshold1
    result = np.zeros_like(matrix[:, 1:])
    transformed = (matrix[:, 1:] >= threshold2).astype(int)
    result[~mask1] = transformed[~mask1]
    return result

def process_predictions(pred_emo):
    pred_emo = torch.nn.functional.softmax(pred_emo, dim=1).cpu().detach().numpy()
    pred_emo = transform_matrix(pred_emo).tolist()
    return pred_emo

def get_metadata(video_path: str, segment_length: int, image_processor: None, image_feature_extractor: None, device: None) -> pd.DataFrame:
    """Основная функция: получает метаданные для видео."""
    if hasattr(body_detector.predictor, 'trackers'):
        body_detector.predictor.trackers[0].reset()
    
    cap = cv2.VideoCapture(video_path)
    video_name = os.path.basename(video_path)
    w, h, fps, total_frames = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS, cv2.CAP_PROP_FRAME_COUNT))
    need_frames = select_uniform_frames(list(range(total_frames)), segment_length)
    
    counter = 0
    embeds = []

    body_list = []
    face_list = []
    
    while True:
        ret, im0 = cap.read()
        if not ret:
            break

        if counter in need_frames:
            # Детекция всех лиц
            preprocessed_body = []
            preprocessed_face = []
            face_results = face_detector.process(cv2.cvtColor(im0, cv2.COLOR_BGR2RGB))
            # Детекция всех тел
            body_results = body_detector.track(im0, persist=True, imgsz=640, conf=0.01, iou=0.5, 
                                             augment=False, device=0, verbose=False)

            # Случай 1: Есть лица — обрабатываем каждое
            if face_results.detections:
                for face_idx, detection in enumerate(face_results.detections):
                    # Координаты лица
                    bbox = detection.location_data.relative_bounding_box
                    x1, y1 = max(int(bbox.xmin * w), 0), max(int(bbox.ymin * h), 0)
                    x2, y2 = min(int((bbox.xmin + bbox.width) * w), w), min(int((bbox.ymin + bbox.height) * h), h)
                    face_bbox = (x1, y1, x2, y2)
                    face_center = ((x1 + x2) // 2, (y1 + y2) // 2)

                    # Ищем тело, содержащее центр лица
                    body_bbox = None
                    body_id = -1
                    if body_results and len(body_results[0].boxes) > 0:
                        for box in body_results[0].boxes:
                            box_coords = box.xyxy.int().cpu().numpy()[0]
                            if (box_coords[0] <= face_center[0] <= box_coords[2] and 
                                box_coords[1] <= face_center[1] <= box_coords[3]):
                                body_bbox = box_coords
                                body_id = box.id.int().cpu().item() if box.id else -1
                                break

                    # Предобработка
                    face_roi = im0[y1:y2, x1:x2]
                    draw_box(im0, [x1, y1, x2, y2])
                    draw_box(im0, [body_bbox[0], body_bbox[1], body_bbox[2], body_bbox[3]])
                    preprocessed_face = image_processing(face_roi, image_processor) if face_roi.size > 0 else None
                    
                    if body_bbox is not None:
                        body_roi = im0[body_bbox[1]:body_bbox[3], body_bbox[0]:body_bbox[2]]
                        preprocessed_body = image_processing(body_roi, image_processor) if body_roi.size > 0 else None
                    else:
                        preprocessed_body = []

                    # Сохраняем результат
                    embeds.append([
                        video_name, counter, body_id,
                        x1, y1, x2, y2,
                        body_bbox[0] if body_bbox is not None else None,
                        body_bbox[1] if body_bbox is not None else None,
                        body_bbox[2] if body_bbox is not None else None,
                        body_bbox[3] if body_bbox is not None else None,
                        # preprocessed_face,
                        # preprocessed_body
                    ])
                    # print(preprocessed_body.shape)
                    # print(preprocessed_face.shape)
                    if preprocessed_body.shape[0] > 0:
                        body_list.append(preprocessed_body)
                    if preprocessed_face.shape[0] > 0:
                        face_list.append(preprocessed_face)
                    

            # Случай 2: Лиц нет — берём самое большое тело
            elif body_results and len(body_results[0].boxes) > 0:
                largest_body = max(
                    body_results[0].boxes,
                    key=lambda box: (box.xyxy[0,2] - box.xyxy[0,0]) * (box.xyxy[0,3] - box.xyxy[0,1])
                )
                body_coords = largest_body.xyxy.int().cpu().numpy()[0]
                body_id = largest_body.id.int().cpu().item() if largest_body.id else -1

                # Предобработка тела
                body_roi = im0[body_coords[1]:body_coords[3], body_coords[0]:body_coords[2]]
                preprocessed_body = preprocess_body(body_roi) if body_roi.size > 0 else []

                embeds.append([
                    video_name, counter, body_id,
                    None, None, None, None,  # Нет лица
                    body_coords[0], body_coords[1], body_coords[2], body_coords[3],
                    # None,  # Нет лица
                    # preprocessed_body
                ])

                if preprocessed_body.shape[0] > 0:
                    body_list.append(preprocessed_body)
                if preprocessed_face.shape[0] > 0:
                    face_list.append(preprocessed_face)

            plt.imshow(cv2.cvtColor(im0, cv2.COLOR_BGR2RGB))
            plt.show()

        counter += 1
        torch.cuda.empty_cache()

    cap.release()

    body_list = torch.cat(body_list, dim=0)
    body_feature = image_feature_extractor.extract(body_list).to(device)

    face_list = torch.cat(face_list, dim=0)
    face_feature = image_feature_extractor.extract(face_list).to(device)
    
    df = pd.DataFrame(embeds, columns=[
        "video_name", "frame", "person_id",
        "face_x1", "face_y1", "face_x2", "face_y2",
        "body_x1", "body_y1", "body_x2", "body_y2",
        # "preprocessed_face", "preprocessed_body"
    ])
    return df, body_feature, face_feature

In [7]:
mp_face_detection = mp.solutions.face_detection
face_detector = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.6)
body_detector = YOLO('extractors/body/best.pt')
image_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
config_body = ConfigLoader("inference_config_body.toml")
config_face = ConfigLoader("inference_config_face.toml")
# image_feature_extractor = PretrainedImageEmbeddingExtractor(config_body)
image_feature_extractor = PretrainedImageEmbeddingExtractor(device="cuda")
# Models can download from https://drive.google.com/drive/folders/1APMtC4LXjuW9behd2TxVXz0DsjQKAgRR?usp=sharing

body_model = get_fusion_model(config_body, 'cuda')
face_model = get_fusion_model(config_face, 'cuda')
# results_clip_body_true_mamba_fusiontransformer_2025-06-27_16-10-57/metrics_by_epoch/metrics_epochlog_FusionTransformer_num_transformer_heads_16_20250627_183039_timestamp/best_model_dev.pt
body_fusion_model_path = 'extractors/body/clip_body_mamba_transformer_fusion_model.pt'
# results_fusiontransformer_2025-07-03_09-41-13/metrics_by_epoch/metrics_epochlog_FusionTransformer_tr_layer_number_3_20250703_124848_timestamp/best_model_dev.pt
face_fusion_model_path = 'extractors/face/clip_face_mamba_transformer_fusion_model.pt'

body_state = torch.load(body_fusion_model_path, map_location='cuda')
body_model.load_state_dict(body_state)

face_state = torch.load(face_fusion_model_path, map_location='cuda')
face_model.load_state_dict(face_state)

RuntimeError: Error(s) in loading state_dict for FusionTransformer:
	size mismatch for emo_proj.0.weight: copying a param with shape torch.Size([1024, 256]) from checkpoint, the shape in current model is torch.Size([256, 256]).
	size mismatch for emo_proj.0.bias: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for emo_proj.1.weight: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for emo_proj.1.bias: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for per_proj.0.weight: copying a param with shape torch.Size([1024, 1024]) from checkpoint, the shape in current model is torch.Size([256, 1024]).
	size mismatch for per_proj.0.bias: copying a param with shape torch.Size([1024]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch f

In [56]:
import torch

# путь к твоему файлу .pt
pt_path = r"C:/Prgrm/ICLR26/features/cmu_mosei/dev/face/face__clipv-openai-clip-vit-base-patch32__frames30__img224__avg1__pv-v1/feats_seed42_subset4_avg1.pt"

# загружаем через PyTorch
data = torch.load(pt_path, map_location="cpu")

print("Тип верхнего объекта:", type(data))

# посмотрим header
print("\n----- HEADER -----")
if "header" in data:
    for k, v in data["header"].items():
        print(f"{k}: {v}")

# посмотрим data
print("\n----- DATA -----")
if "data" in data:
    sample_keys = list(data["data"].keys())
    print("Количество сэмплов:", len(sample_keys))
    print("Пример ключей:", sample_keys[:5])  # первые 5 ключей

    # берём первый сэмпл
    first_key = sample_keys[0]
    print("\nПервый ключ:", first_key)

    sample = data["data"][first_key]

    # показываем, что внутри (обычно mean / std)
    for stat_name, tensor in sample.items():
        if torch.is_tensor(tensor):
            print(f"{stat_name}: тензор формы {tensor.shape}")
            # первые 10 чисел для наглядности
            print("  первые значения:", tensor[:10])
        else:
            print(f"{stat_name}: {type(tensor)}")


Тип верхнего объекта: <class 'dict'>

----- HEADER -----
mod: face
extractor_fp: clipv:openai/clip-vit-base-patch32
avg: True
frames: 30
img: 224
text_col: None
pre_v: v1

----- DATA -----
Количество сэмплов: 4
Пример ключей: ['--qXJuDtHPw_23.1990_30.3250', '-571d8cVauQ_0.0000_4.9920', '-571d8cVauQ_21.8420_28.3090', '-571d8cVauQ_69.8930_85.0000']

Первый ключ: --qXJuDtHPw_23.1990_30.3250
mean: тензор формы torch.Size([512])
  первые значения: tensor([-0.0106, -0.0707, -0.3738,  0.1331,  0.1042, -0.0987,  0.3389,  0.4817,
         0.1298,  0.3130])


In [77]:
import torch

# путь к твоему файлу .pt
pt_path = r"C:/Prgrm/ICLR26/features/cmu_mosei/dev/audio/audio__clapa-laion-clap-htsat-fused__avg-mean_std__pv-v1/feats_seed42_subset0_avg-mean_std.pt"

# загружаем через PyTorch
data = torch.load(pt_path, map_location="cpu")

print("Тип верхнего объекта:", type(data))

# посмотрим header
print("\n----- HEADER -----")
if "header" in data:
    for k, v in data["header"].items():
        print(f"{k}: {v}")

# посмотрим data
print("\n----- DATA -----")
if "data" in data:
    sample_keys = list(data["data"].keys())
    print("Количество сэмплов:", len(sample_keys))
    print("Пример ключей:", sample_keys[:5])  # первые 5 ключей

    # берём первый сэмпл
    first_key = sample_keys[0]
    print("\nПервый ключ:", first_key)

    sample = data["data"][first_key]

    # показываем, что внутри (обычно mean / std)
    for stat_name, tensor in sample.items():
        if torch.is_tensor(tensor):
            print(f"{stat_name}: тензор формы {tensor.shape}")
            # первые 10 чисел для наглядности
            print("  первые значения:", tensor[:10])
        else:
            print(f"{stat_name}: {type(tensor)}")


Тип верхнего объекта: <class 'dict'>

----- HEADER -----
mod: audio
extractor_fp: clapa:laion/clap-htsat-fused
avg: mean
frames: 0
img: 0
text_col: None
pre_v: v1

----- DATA -----
Количество сэмплов: 1861
Пример ключей: ['--qXJuDtHPw_23.1990_30.3250', '-571d8cVauQ_0.0000_4.9920', '-571d8cVauQ_21.8420_28.3090', '-571d8cVauQ_69.8930_85.0000', '-I_e4mIh0yE_16.8030_25.4695']

Первый ключ: --qXJuDtHPw_23.1990_30.3250
mean: тензор формы torch.Size([512])
  первые значения: tensor([-0.0929, -0.0156,  0.0312, -0.0003,  0.0519, -0.0277, -0.0375, -0.1216,
        -0.0494,  0.0514])


In [70]:
import torch

# путь к твоему файлу .pt
pt_path = r"C:/Prgrm/ICLR26/features/cmu_mosei/dev/audio/audio__clapa-laion-clap-htsat-fused__avg-mean_std__pv-v1/feats_seed42_subset5_avg-mean_std.pt"

# загружаем через PyTorch
data = torch.load(pt_path, map_location="cpu")

print("Тип верхнего объекта:", type(data))

# посмотрим header
print("\n----- HEADER -----")
if "header" in data:
    for k, v in data["header"].items():
        print(f"{k}: {v}")

# посмотрим data
print("\n----- DATA -----")
if "data" in data:
    sample_keys = list(data["data"].keys())
    print("Количество сэмплов:", len(sample_keys))
    print("Пример ключей:", sample_keys[:5])  # первые 5 ключей

    # берём первый сэмпл
    first_key = sample_keys[0]
    print("\nПервый ключ:", first_key)

    sample = data["data"][first_key]

    # показываем, что внутри (обычно mean / std)
    for stat_name, tensor in sample.items():
        if torch.is_tensor(tensor):
            print(f"{stat_name}: тензор формы {tensor.shape}")
            # первые 10 чисел для наглядности
            print("  первые значения:", tensor[:10])
        else:
            print(f"{stat_name}: {type(tensor)}")


Тип верхнего объекта: <class 'dict'>

----- HEADER -----
mod: audio
extractor_fp: clapa:laion/clap-htsat-fused
avg: mean_std
frames: 0
img: 0
text_col: None
pre_v: v1

----- DATA -----
Количество сэмплов: 5
Пример ключей: ['--qXJuDtHPw_23.1990_30.3250', '-571d8cVauQ_0.0000_4.9920', '-571d8cVauQ_21.8420_28.3090', '-571d8cVauQ_69.8930_85.0000', '-I_e4mIh0yE_3.1440_10.5700']

Первый ключ: --qXJuDtHPw_23.1990_30.3250
mean: тензор формы torch.Size([768])
  первые значения: tensor([ 0.5888, -0.0429,  0.1800, -0.2739, -0.7886,  0.7882,  0.0318,  0.1876,
        -0.4165, -0.8684])
std: тензор формы torch.Size([768])
  первые значения: tensor([0.1291, 0.4231, 0.9009, 0.6434, 0.0283, 0.1644, 0.0821, 0.0370, 0.6012,
        0.1113])


In [60]:
import torch

# путь к твоему файлу .pt
pt_path = r"C:/Prgrm/ICLR26/features/cmu_mosei/dev/text/text__clapt-laion-clap-htsat-fused__avg1__pv-v1/feats_seed42_subset3_avg1.pt"

# загружаем через PyTorch
data = torch.load(pt_path, map_location="cpu")

print("Тип верхнего объекта:", type(data))

# посмотрим header
print("\n----- HEADER -----")
if "header" in data:
    for k, v in data["header"].items():
        print(f"{k}: {v}")

# посмотрим data
print("\n----- DATA -----")
if "data" in data:
    sample_keys = list(data["data"].keys())
    print("Количество сэмплов:", len(sample_keys))
    print("Пример ключей:", sample_keys[:5])  # первые 5 ключей

    # берём первый сэмпл
    first_key = sample_keys[0]
    print("\nПервый ключ:", first_key)

    sample = data["data"][first_key]

    # показываем, что внутри (обычно mean / std)
    for stat_name, tensor in sample.items():
        if torch.is_tensor(tensor):
            print(f"{stat_name}: тензор формы {tensor.shape}")
            # первые 10 чисел для наглядности
            print("  первые значения:", tensor[:10])
        else:
            print(f"{stat_name}: {type(tensor)}")


Тип верхнего объекта: <class 'dict'>

----- HEADER -----
mod: text
extractor_fp: clapt:laion/clap-htsat-fused
avg: True
frames: 0
img: 0
text_col: None
pre_v: v1

----- DATA -----
Количество сэмплов: 3
Пример ключей: ['--qXJuDtHPw_23.1990_30.3250', '-571d8cVauQ_0.0000_4.9920', '-571d8cVauQ_21.8420_28.3090']

Первый ключ: --qXJuDtHPw_23.1990_30.3250
mean: тензор формы torch.Size([768])
  первые значения: tensor([ 0.1183,  0.0298,  0.1684,  0.8365, -0.0719,  0.0729, -0.1034,  0.1622,
        -0.3237, -0.6614])
std: тензор формы torch.Size([768])
  первые значения: tensor([0.3033, 0.1507, 0.1535, 0.4128, 0.3745, 0.2035, 0.1853, 0.4333, 0.2242,
        0.2408])


In [62]:
import torch

# путь к твоему файлу .pt
pt_path = r"C:/Prgrm/ICLR26/features/cmu_mosei/dev/behavior/behavior__clipt-openai-clip-vit-base-patch32__col-text_llm__avg1__pv-v1/feats_seed42_subset4_avg1.pt"

# загружаем через PyTorch
data = torch.load(pt_path, map_location="cpu")

print("Тип верхнего объекта:", type(data))

# посмотрим header
print("\n----- HEADER -----")
if "header" in data:
    for k, v in data["header"].items():
        print(f"{k}: {v}")

# посмотрим data
print("\n----- DATA -----")
if "data" in data:
    sample_keys = list(data["data"].keys())
    print("Количество сэмплов:", len(sample_keys))
    print("Пример ключей:", sample_keys[:5])  # первые 5 ключей

    # берём первый сэмпл
    first_key = sample_keys[0]
    print("\nПервый ключ:", first_key)

    sample = data["data"][first_key]

    # показываем, что внутри (обычно mean / std)
    for stat_name, tensor in sample.items():
        if torch.is_tensor(tensor):
            print(f"{stat_name}: тензор формы {tensor.shape}")
            # первые 10 чисел для наглядности
            print("  первые значения:", tensor[:10])
        else:
            print(f"{stat_name}: {type(tensor)}")


Тип верхнего объекта: <class 'dict'>

----- HEADER -----
mod: behavior
extractor_fp: clipt:openai/clip-vit-base-patch32
avg: True
frames: 0
img: 0
text_col: text_llm
pre_v: v1

----- DATA -----
Количество сэмплов: 4
Пример ключей: ['--qXJuDtHPw_23.1990_30.3250', '-571d8cVauQ_0.0000_4.9920', '-571d8cVauQ_21.8420_28.3090', '-571d8cVauQ_69.8930_85.0000']

Первый ключ: --qXJuDtHPw_23.1990_30.3250
mean: тензор формы torch.Size([512])
  первые значения: tensor([ 0.0166,  0.1318,  0.1382, -0.2897, -0.2518, -0.0270,  0.0190, -0.1784,
         0.0061,  0.0829])


In [66]:
import pickle
import pandas as pd

file_path = 'C:/Prgrm/ICLR26/features/cmu_mosei/train/meta_seed42_subset5.pickle'

with open(file_path, 'rb') as f:
    data = pickle.load(f)

print(f"Тип загруженного объекта: {type(data)}\n")

# Если это DataFrame
if isinstance(data, pd.DataFrame):
    print("DataFrame — первые строки:")
    print(data.head())

# Если это словарь
elif isinstance(data, dict):
    print("Словарь — первые ключи и значения:")
    for i, (key, value) in enumerate(data.items()):
        print(f"[{i}] Ключ: {key} — Тип значения: {type(value)}")
        print(f"  Значение: {repr(value)[:1000]}")
        print("-" * 80)
        if i >= 4:
            break

# Если это список или кортеж
elif isinstance(data, (list, tuple)):
    print(f"{type(data).__name__} — первые элементы:")
    for i, item in enumerate(data[:5]):
        print(f"[{i}] — Тип: {type(item)}")
        print(f"  Значение: {repr(item)[:1000]}")
        print("-" * 80)

# Всё остальное
else:
    print("Неизвестный тип. Показываю первые 1000 символов:")
    print(repr(data)[:1000])


Тип загруженного объекта: <class 'list'>

list — первые элементы:
[0] — Тип: <class 'dict'>
  Значение: {'sample_name': '-3g5yACwYnA_119.9190_125.2990', 'video_path': 'E:/CMU-MOSEI//video/train/-3g5yACwYnA_119.9190_125.2990.mp4', 'audio_path': 'E:/CMU-MOSEI//audio/train/-3g5yACwYnA_119.9190_125.2990.wav', 'labels': {'emotion': tensor([1., 0., 0., 0., 0., 0., 0.]), 'personality': tensor([nan, nan, nan, nan, nan]), 'ah': tensor(nan)}}
--------------------------------------------------------------------------------
[1] — Тип: <class 'dict'>
  Значение: {'sample_name': '-3g5yACwYnA_13.6315_27.0310', 'video_path': 'E:/CMU-MOSEI//video/train/-3g5yACwYnA_13.6315_27.0310.mp4', 'audio_path': 'E:/CMU-MOSEI//audio/train/-3g5yACwYnA_13.6315_27.0310.wav', 'labels': {'emotion': tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.5000, 0.0000]), 'personality': tensor([nan, nan, nan, nan, nan]), 'ah': tensor(nan)}}
--------------------------------------------------------------------------------
[2] — Ти

In [67]:
import pickle
import pandas as pd

file_path = 'C:/Prgrm/ICLR26/features/cmu_mosei/train/meta_seed42_subset0.pickle'

with open(file_path, 'rb') as f:
    data = pickle.load(f)

print(f"Тип загруженного объекта: {type(data)}\n")

# Если это DataFrame
if isinstance(data, pd.DataFrame):
    print("DataFrame — первые строки:")
    print(data.head())

# Если это словарь
elif isinstance(data, dict):
    print("Словарь — первые ключи и значения:")
    for i, (key, value) in enumerate(data.items()):
        print(f"[{i}] Ключ: {key} — Тип значения: {type(value)}")
        print(f"  Значение: {repr(value)[:1000]}")
        print("-" * 80)
        if i >= 4:
            break

# Если это список или кортеж
elif isinstance(data, (list, tuple)):
    print(f"{type(data).__name__} — первые элементы:")
    for i, item in enumerate(data[:5]):
        print(f"[{i}] — Тип: {type(item)}")
        print(f"  Значение: {repr(item)[:1000]}")
        print("-" * 80)

# Всё остальное
else:
    print("Неизвестный тип. Показываю первые 1000 символов:")
    print(repr(data)[:1000])


Тип загруженного объекта: <class 'list'>

list — первые элементы:
[0] — Тип: <class 'dict'>
  Значение: {'sample_name': '-3g5yACwYnA_119.9190_125.2990', 'video_path': 'E:/CMU-MOSEI//video/train/-3g5yACwYnA_119.9190_125.2990.mp4', 'audio_path': 'E:/CMU-MOSEI//audio/train/-3g5yACwYnA_119.9190_125.2990.wav', 'labels': {'emotion': tensor([1., 0., 0., 0., 0., 0., 0.]), 'personality': tensor([nan, nan, nan, nan, nan]), 'ah': tensor(nan)}}
--------------------------------------------------------------------------------
[1] — Тип: <class 'dict'>
  Значение: {'sample_name': '-3g5yACwYnA_13.6315_27.0310', 'video_path': 'E:/CMU-MOSEI//video/train/-3g5yACwYnA_13.6315_27.0310.mp4', 'audio_path': 'E:/CMU-MOSEI//audio/train/-3g5yACwYnA_13.6315_27.0310.wav', 'labels': {'emotion': tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.5000, 0.5000, 0.0000]), 'personality': tensor([nan, nan, nan, nan, nan]), 'ah': tensor(nan)}}
--------------------------------------------------------------------------------
[2] — Ти

In [34]:
import torch
import numpy as np
# Дополнительно: распечатать шейпы признаков по модальностям
print("\n🔎 Шейпы признаков:")
modalities = item.get("features", {})
for mod_name, features in modalities.items():
    print(f"\n[{mod_name.upper()}]")
    for feat_name, feat_val in features.items():
        if isinstance(feat_val, torch.Tensor):
            print(f"  {feat_name}: {feat_val.shape}")
        elif isinstance(feat_val, np.ndarray):
            print(f"  {feat_name}: {feat_val.shape}")
        else:
            print(f"  {feat_name}: not a tensor ({type(feat_val)})")


🔎 Шейпы признаков:

[BODY]
  emotion_logits: torch.Size([7])
  personality_scores: torch.Size([5])
  last_emo_encoder_features: torch.Size([1024])
  last_per_encoder_features: torch.Size([1024])

[FACE]
  emotion_logits: torch.Size([7])
  personality_scores: torch.Size([5])
  last_emo_encoder_features: torch.Size([512])
  last_per_encoder_features: torch.Size([512])

[SCENE]
  emotion_logits: torch.Size([7])
  personality_scores: torch.Size([5])
  last_emo_encoder_features: torch.Size([768])
  last_per_encoder_features: torch.Size([768])

[AUDIO]
  emotion_logits: torch.Size([7])
  personality_scores: torch.Size([5])
  last_emo_encoder_features: torch.Size([256])
  last_per_encoder_features: torch.Size([256])

[TEXT]
  emotion_logits: torch.Size([7])
  personality_scores: torch.Size([5])
  last_emo_encoder_features: torch.Size([256])
  last_per_encoder_features: torch.Size([256])


In [44]:
import pickle

# путь к твоему .pickle файлу
pickle_path = "../features/cmu_mosei_test_seed_42_subset_size_2_average_features_False_feature_norm_False.pickle"

with open(pickle_path, "rb") as f:
    data = pickle.load(f)

# Посмотреть первый элемент
print("🔍 Первый элемент:")
item = data[0]
print(item)



FileNotFoundError: [Errno 2] No such file or directory: '../features/cmu_mosei_test_seed_42_subset_size_2_average_features_False_feature_norm_False.pickle'

In [49]:
import pickle

# путь к твоему .pickle файлу
pickle_path = "../features/fiv2_test_seed_42_subset_size_2_average_features_True_feature_norm_False.pickle"

with open(pickle_path, "rb") as f:
    data = pickle.load(f)

# Посмотреть первый элемент
print("🔍 Первый элемент:")
item = data[0]["features"]
print(item)



🔍 Первый элемент:
{'body': {'emotion_logits': tensor([-1.3183, -0.8238, -0.6552, -0.6306,  1.2407, -0.7158,  1.3891]), 'personality_scores': tensor([0.5326, 0.5365, 0.4849, 0.6321, 0.5423]), 'last_emo_encoder_features': tensor([-3.8351, -2.2751, -1.4746,  ..., -3.7185, -0.7212, -0.6948]), 'last_per_encoder_features': tensor([ 4.4531,  1.6310,  1.1421,  ...,  0.1990, -1.8134, -0.8094])}, 'face': {'emotion_logits': tensor([ 0.3950, -1.3291, -0.8733,  0.8395, -0.5555, -0.5297, -0.3164]), 'personality_scores': tensor([0.5448, 0.5832, 0.4877, 0.6371, 0.5482]), 'last_emo_encoder_features': tensor([-7.8037e+00,  7.3685e-01, -6.7197e-01,  5.2608e+00,  2.7093e+00,
        -8.5908e-01,  7.0660e-01,  1.8818e+00, -7.1252e-02,  1.1847e-01,
        -2.0084e+00,  6.3778e-01,  2.5741e+00, -1.2668e+00, -2.0097e+00,
        -7.8275e+00,  3.3383e+00, -3.7198e+00, -1.6923e+00, -3.8017e-01,
         8.0145e+00, -9.3273e-01, -6.2941e-01,  7.6822e-01,  3.5799e+00,
        -5.1096e+00,  1.1017e+00, -4.3097e+0

In [43]:
import torch
import numpy as np
# Дополнительно: распечатать шейпы признаков по модальностям
print("\n🔎 Шейпы признаков:")
modalities = item.get("features", {})
for mod_name, features in modalities.items():
    print(f"\n[{mod_name.upper()}]")
    for feat_name, feat_val in features.items():
        if isinstance(feat_val, torch.Tensor):
            print(f"  {feat_name}: {feat_val.shape}")
        elif isinstance(feat_val, np.ndarray):
            print(f"  {feat_name}: {feat_val.shape}")
        else:
            print(f"  {feat_name}: not a tensor ({type(feat_val)})")


🔎 Шейпы признаков:

[BODY]
  emotion_logits: torch.Size([7])
  personality_scores: torch.Size([5])
  last_emo_encoder_features: torch.Size([1024])
  last_per_encoder_features: torch.Size([1024])

[FACE]
  emotion_logits: torch.Size([7])
  personality_scores: torch.Size([5])
  last_emo_encoder_features: torch.Size([512])
  last_per_encoder_features: torch.Size([512])

[SCENE]
  emotion_logits: torch.Size([7])
  personality_scores: torch.Size([5])
  last_emo_encoder_features: torch.Size([768])
  last_per_encoder_features: torch.Size([768])

[AUDIO]
  emotion_logits: torch.Size([7])
  personality_scores: torch.Size([5])
  last_emo_encoder_features: torch.Size([256])
  last_per_encoder_features: torch.Size([256])

[TEXT]
  emotion_logits: torch.Size([7])
  personality_scores: torch.Size([5])
  last_emo_encoder_features: torch.Size([256])
  last_per_encoder_features: torch.Size([256])


In [2]:
print(type(data))
print(f"Элементов в списке: {len(data)}")

<class 'list'>
Элементов в списке: 10


In [3]:
import pandas as pd
import os


def clean_csv_to_copy(csv_path: str, output_path: str):
    df = pd.read_csv(csv_path)

    # Удаляем неназванные колонки
    unnamed_cols = [col for col in df.columns if col.startswith("Unnamed") or col.strip() == ""]
    if unnamed_cols:
        print(f"Удаляем колонку/и: {unnamed_cols}")
        df = df.drop(columns=unnamed_cols)

    # Чистим расширения в video_name
    if 'video_name' not in df.columns:
        raise ValueError("Файл без 'video_name'. Ты что, решил пошутить?")
    
    df['video_name'] = df['video_name'].apply(lambda x: os.path.splitext(str(x))[0])

    # Сохраняем в новый файл
    df.to_csv(output_path, index=False)
    print(f"Готово. Чистый CSV сохранён сюда: {output_path}")
    
# Пример использования
csv_path = "E:/FirstImpressionsV2/dev_FIv2.csv"
output_path = "E:/FirstImpressionsV2/dev_full.csv"
clean_csv_to_copy(csv_path, output_path)


Удаляем колонку/и: ['Unnamed: 0']
Готово. Чистый CSV сохранён сюда: E:/FirstImpressionsV2/dev_full.csv


In [4]:
csv_path = "E:/FirstImpressionsV2/test_FIv2.csv"
output_path = "E:/FirstImpressionsV2/test_full.csv"
clean_csv_to_copy(csv_path, output_path)

Удаляем колонку/и: ['Unnamed: 0']
Готово. Чистый CSV сохранён сюда: E:/FirstImpressionsV2/test_full.csv


In [5]:
csv_path = "E:/FirstImpressionsV2/train_FIv2.csv"
output_path = "E:/FirstImpressionsV2/train_full.csv"
clean_csv_to_copy(csv_path, output_path)

Удаляем колонку/и: ['Unnamed: 0']
Готово. Чистый CSV сохранён сюда: E:/FirstImpressionsV2/train_full.csv


In [16]:
import torch

checkpoint = torch.load("../modalities/text/checkpoints/Mamba_Transformer_bge-small_fusion.pt", map_location="cpu")

print(checkpoint.keys())

odict_keys(['emo_model.emo_proj.0.weight', 'emo_model.emo_proj.0.bias', 'emo_model.emo_proj.1.weight', 'emo_model.emo_proj.1.bias', 'emo_model.emotion_encoder.0.in_proj.weight', 'emo_model.emotion_encoder.0.in_proj.bias', 'emo_model.emotion_encoder.0.s_B.weight', 'emo_model.emotion_encoder.0.s_B.bias', 'emo_model.emotion_encoder.0.s_C.weight', 'emo_model.emotion_encoder.0.s_C.bias', 'emo_model.emotion_encoder.0.out_proj.weight', 'emo_model.emotion_encoder.0.out_proj.bias', 'emo_model.emotion_encoder.0.norm.weight', 'emo_model.emotion_encoder.0.norm.bias', 'emo_model.emotion_encoder.1.in_proj.weight', 'emo_model.emotion_encoder.1.in_proj.bias', 'emo_model.emotion_encoder.1.s_B.weight', 'emo_model.emotion_encoder.1.s_B.bias', 'emo_model.emotion_encoder.1.s_C.weight', 'emo_model.emotion_encoder.1.s_C.bias', 'emo_model.emotion_encoder.1.out_proj.weight', 'emo_model.emotion_encoder.1.out_proj.bias', 'emo_model.emotion_encoder.1.norm.weight', 'emo_model.emotion_encoder.1.norm.bias', 'emo_mod